In [2]:
%pylab inline

import os
import logging
import time
import asyncio
import datetime
import glob
import pathlib
from pprint import pprint
from shutil import copyfile

import numpy as np
np.set_printoptions(precision=3)
import matplotlib.pyplot as plt
import seaborn
seaborn.set_style("whitegrid")
import pandas as pd
import h5py

from artiq.master.databases import DeviceDB
from artiq.master.worker_db import DeviceManager
from sipyco.pc_rpc import (Client, AsyncioClient,
                                    BestEffortClient, AutoTarget)
import sipyco.pyon as pyon

Populating the interactive namespace from numpy and matplotlib


In [115]:
# Load experiment prototypes from Q://
base_dir = pathlib.Path(r"//EURIQA-NAS/lab/CompactTrappedIonModule")
prototype_path = base_dir / "ARTIQ/experiment_prototypes"

experiments = dict()
for i, exp in enumerate(os.listdir(str(prototype_path))):
    file = h5py.File(str(prototype_path / exp),"r")                    
    name = exp.split(".",1)[0]
    experiments[name] = dict()
    experiments[name]["expid"] = pyon.decode(file["expid"].value) 
    
print("Available experiments are:")
for i in list(experiments.keys()):
    print(i)

Available experiments are:
Autoload_01ions
CheckDZ_15ions
Check_center_15ions
Circuit_15ions
Autoload_15ions
Check_radial_15ions
Check_axial_15ions
Check_high_radial_15ions
Check_alive_15ions


In [3]:
# Load Circuit List
circuit_path = base_dir / "Data/Circuits/2019/06_2019/27_06_2019_RB_groundIdle/single_q_83"
circuit_list = []
for filename in circuit_path.glob('*/circuit*'):
    # Ignore Qubits 1 and 15  (zero-indexed)
    # if not ("qubit_14" in str(filename) or "qubit_00" in str(filename)) :
    if "qubit_08" in str(filename) :
        circuit_list.append(filename)

num_circuits = len(circuit_list)
print("Number of Circuits Loaded: {0}".format(num_circuits))

batch_size = 10
num_batches = num_circuits/batch_size
if num_batches%1 > 0:
    num_batches =+1
num_batches = int(np.floor(num_batches))

batch_list = []
for i in range(num_batches):
    if i< num_batches-1:
        batch_list.append(circuit_list[i*batch_size:i*batch_size+batch_size])
    else:
        batch_list.append(circuit_list[i*batch_size:])
print("Attemping to run as {0} batchs of size {1}".format(num_batches,batch_size))


Number of Circuits Loaded: 50
Attemping to run as 5 batchs of size 10


In [4]:
# Specify folders 
artiq_data_path =  base_dir / "Data/artiq_data/2019-07-01"
save_folder = base_dir / "Data/2019/06_2019/30_06_2019/rb_singlequbit/"
if not os.path.exists(str(save_folder)):
    os.mkdir(str(save_folder))

In [126]:
# let's connect to the master
master_ip = "192.168.78.144"
schedule, exps, datasets = [
    Client(master_ip, 3251, "master_" + i) for i in
    "schedule experiment_db dataset_db".split()]


In [6]:
failed_batches =[]

In [ ]:
for i, ibatch in enumerate(batch_list[0:9]):
    batch_rid = []

    # Batch submit
    for icircuit in ibatch:
        experiments["Circuit_15ions"]["expid"]["arguments"]["path_to_circuit"] = str(icircuit)
        rid = schedule.submit(pipeline_name="main",
                      expid=experiments["Circuit_15ions"]['expid'],
                      priority=0,
                      due_date=None,
                      flush=False)
        batch_rid.append(rid)
        
    # Wait for them all to finish
    while rid in schedule.get_status():
        time.sleep(0.1)
    
    #Check to see HDF files saved and no experiment failed
    success = True
    data_list = []
    for irid in batch_rid:
        data = list(artiq_data_path.glob("**/{:09d}*".format(irid)))
        if len(data)==0:
            success = False
        else:
            data_list.append(data)
            
    if success is True:
        print("Batch {0} succeeded!".format(i))
        batch_folder = save_folder / "batch_{:03d}".format(i)

        print("Copying data and circuits to {0}".format(str(batch_folder)))
        if not os.path.exists(str(batch_folder)):
            os.mkdir(str(batch_folder))
            
        for i, irid in enumerate(batch_rid) :
            copyfile(str(ibatch[i]), str(batch_folder / ("{:09d}_".format(irid) + ibatch[i].name)))
            copyfile(str(data_list[i][0]), str(batch_folder / data_list[i][0].name))        
        
    else:
        print("Batch {0} Failed. Adding batch number to failed_batches list".format(i))
        failed_batches.append(i)
            
    input("Press Enter to submit the next batch")

        

Batch 0 succeeded!
Copying data and circuits to \\EURIQA-NAS\lab\CompactTrappedIonModule\Data\2019\06_2019\30_06_2019\rb_singlequbit\batch_000
Press Enter to submit the next batch
Batch 1 succeeded!
Copying data and circuits to \\EURIQA-NAS\lab\CompactTrappedIonModule\Data\2019\06_2019\30_06_2019\rb_singlequbit\batch_001
Press Enter to submit the next batch
Batch 2 succeeded!
Copying data and circuits to \\EURIQA-NAS\lab\CompactTrappedIonModule\Data\2019\06_2019\30_06_2019\rb_singlequbit\batch_002
Press Enter to submit the next batch
Batch 3 Failed. Adding batch number to failed_batches list
Press Enter to submit the next batch
Batch 4 Failed. Adding batch number to failed_batches list


In [ ]:
# OTHER CELLS THAT MAY OR MAY NOT BE USEFUL BELOW

In [375]:
# Submitting Loading Scripts for 15 ion
rid = schedule.submit(pipeline_name="main",
                      expid=experiments["Autoload_15ions"]['expid'],
                      priority=0,
                      due_date=None,
                      flush=False)


In [415]:
# Submit a single Circuit
rid = schedule.submit(pipeline_name="main",
                      expid=experiments["Circuit_15ions"]['expid'],
                      priority=0,
                      due_date=None,
                      flush=False)


In [274]:
# I can pull globals 
datasets.get("monitor.Lost_Ions.Collision")

# And I can set them 
datasets.set("monitor.Lost_Ions.Collision", False)